# 품질검사성적서 등록 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사성적서 등록 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사성적서 등록 목록/selectIoPmQtscList.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [5]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")

In [6]:
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

74882

In [7]:
resultDfMerged = pd.DataFrame()

In [8]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endYr

'2022'

In [9]:
# 기준년도 산정
# 2001년 부터 데이터 존재
startYr = "2001"
dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
# dateRange

## 품질검사성적서 등록정보 전용 스크랩 함수

In [10]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal",x=-1):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount[x] == 0:
                        g_totalCount[x] = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [11]:
numOfThread = 500

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))

g_totalCount = list(range(numOfThread))

In [12]:
def threadReq(startIdx, endIdx,x):
    numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
    resultDf = pd.DataFrame()
    for i in range(startIdx,endIdx):
        searchCstrNm = dfCombi.loc[i].cstrnm
        for j  in dateRange:
            pageNo = 1
            print(searchCstrNm," ",j)
            global g_totalCount
            g_totalCount[x] = 0
            while True:
                BASEPARAM={"serviceKey":APIKEY,"searchCstrNm": searchCstrNm, "searchRpcdIsBgDt": str(j)+"0101","searchRpcdIsEdDt": str(j)+"1231", "sortField": "tstBgDt","pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
                print("BASEPARAM ",BASEPARAM)
                resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY, "jsonabnormal",x)
                if resultDf.empty: # 정상 데이터가 없는 경우
                    print("No Data")
                    break
                else:
                    resultDfMerged[x] = resultDfMerged[x].append(resultDf) 
                    g_totalCount[x] -= numOfRows
                    print("x : ",x," g_totalCount : ", g_totalCount[x], " numOfRows : ", numOfRows)
                    if(g_totalCount[x] <= 0):
                        break
                    pageNo += 1

In [13]:
import threading

resultDfLastMerged = pd.DataFrame()
section = list(range(0, dfCombiLen, int(dfCombiLen/numOfThread))) # we have "numåOfThread +1" sections
section

[0,
 149,
 298,
 447,
 596,
 745,
 894,
 1043,
 1192,
 1341,
 1490,
 1639,
 1788,
 1937,
 2086,
 2235,
 2384,
 2533,
 2682,
 2831,
 2980,
 3129,
 3278,
 3427,
 3576,
 3725,
 3874,
 4023,
 4172,
 4321,
 4470,
 4619,
 4768,
 4917,
 5066,
 5215,
 5364,
 5513,
 5662,
 5811,
 5960,
 6109,
 6258,
 6407,
 6556,
 6705,
 6854,
 7003,
 7152,
 7301,
 7450,
 7599,
 7748,
 7897,
 8046,
 8195,
 8344,
 8493,
 8642,
 8791,
 8940,
 9089,
 9238,
 9387,
 9536,
 9685,
 9834,
 9983,
 10132,
 10281,
 10430,
 10579,
 10728,
 10877,
 11026,
 11175,
 11324,
 11473,
 11622,
 11771,
 11920,
 12069,
 12218,
 12367,
 12516,
 12665,
 12814,
 12963,
 13112,
 13261,
 13410,
 13559,
 13708,
 13857,
 14006,
 14155,
 14304,
 14453,
 14602,
 14751,
 14900,
 15049,
 15198,
 15347,
 15496,
 15645,
 15794,
 15943,
 16092,
 16241,
 16390,
 16539,
 16688,
 16837,
 16986,
 17135,
 17284,
 17433,
 17582,
 17731,
 17880,
 18029,
 18178,
 18327,
 18476,
 18625,
 18774,
 18923,
 19072,
 19221,
 19370,
 19519,
 19668,
 19817,
 1996

In [14]:
# dfCombiLen = 6 # for test
if dfCombiLen < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []
    idx = 0
    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else dfCombiLen - 1
        print("start ",startIdx, " end ",endIdx)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,idx))
        idx += 1
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



start  0  end  148
start  149  end  297
start  298  end  446
start  447  end  595
start  596  end  744
start  745  end  893
start  894  end  1042
start  1043  end  1191
start  1192  end  1340
start  1341  end  1489
start  1490  end  1638
start  1639  end  1787
start  1788  end  1936
start  1937  end  2085
start  2086  end  2234
start  2235  end  2383
start  2384  end  2532
start  2533  end  2681
start  2682  end  2830
start  2831  end  2979
start  2980  end  3128
start  3129  end  3277
start  3278  end  3426
start  3427  end  3575
start  3576  end  3724
start  3725  end  3873
start  3874  end  4022
start  4023  end  4171
start  4172  end  4320
start  4321  end  4469
start  4470  end  4618
start  4619  end  4767
start  4768  end  4916
start  4917  end  5065
start  5066  end  5214
start  5215  end  5363
start  5364  end  5512
start  5513  end  5661
start  5662  end  5810
start  5811  end  5959
start  5960  end  6108
start  6109  end  6257
start  6258  end  6406
start  6407  end  6555
sta

start  70924  end  71072
start  71073  end  71221
start  71222  end  71370
start  71371  end  71519
start  71520  end  71668
start  71669  end  71817
start  71818  end  71966
start  71967  end  72115
start  72116  end  72264
start  72265  end  72413
start  72414  end  72562
start  72563  end  72711
start  72712  end  72860
start  72861  end  73009
start  73010  end  73158
start  73159  end  73307
start  73308  end  73456
start  73457  end  73605
start  73606  end  73754
start  73755  end  73903
start  73904  end  74052
start  74053  end  74201
start  74202  end  74350
start  74351  end  74881
현대건설(주)   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
삼성물산(주)삼성건설   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '삼성물산(주)삼성건설', 'searchRpcdIsBgDt'

세전종합건설(주)/동서종합건설(주)(주)삼일토건  씨앤씨 종합건설(주)   2001DECO C&C (주)덕재건설 2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '세전종합건설(주)/동서종합건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
   1 page scraping start 
 2001
BASEPARAM  
 2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '씨앤씨 종합건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
BASEPARAM 삼성물산㈜ 외 4개社{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)덕재건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)삼일토건', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 

(주)삼성중공업건설유진 건설
  1 page scraping start  
 2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)삼성중공업건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start미래이엔씨(주)   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '미래이엔씨(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

 현대건설(주),한진중공업(주),고려개발(주) 이상3개사   2001현대오일뱅크2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주),한진중공업(주),고려개발(주) 이상3개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'} 
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '유진 건설', 'searchRpcdIsBgDt': '20010101', 'sear

 금호건설(주)외 7개사    2001
BASEPARAM  2001(주)건융
{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '금호건설(주)외 7개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)건융', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page scraping startBASEPARAM (주)뉴영진   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)뉴영진', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

 현대삼호중공업(주)/ (주)코스틸산업{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)청호건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': '

...

Expecting value: line 1 column 1 (char 0)1 page is empty
No Data
현대건설(주)외 3사   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주)외 3사', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start



Exception in thread Exception in thread Thread-465:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Thread-464:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 870, in run
    self.run()
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-2167087db3d6>", line 15, in threadReq
AttributeError: 'NoneType' object has no attribute 'empty'
AttributeError: 'NoneType' object has no attribute 'empty'
Exception in thread Thread-485:
Traceback (most recent call last):
  File "/Users/waynehwang/opt/anaconda3/lib/python3.8/threadi

1 page is emptyExpecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)

1 page is empty

No Data
㈜부원건설,성우종합건설㈜   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '㈜부원건설,성우종합건설㈜', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
No Data
유퓽종합건설   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '유퓽종합건설', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
Expecting value: line 1 column 1 (char 0)
1 page is empty
No Data
DECO C&C   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': 'DECO C&C', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scrapin

KeyboardInterrupt: 

1 page is empty
No Data
(주)포스코건설,세움종합건설(주)   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)포스코건설,세움종합건설(주)', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start


In [15]:
resultDfLastMerged

""


>totalCount 11
0 no pageNo
x :  5  g_totalCount :  -989  numOfRows :  1000
명화건설(주)   2016
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '명화건설(주)', 'searchRpcdIsBgDt': '20160101', 'searchRpcdIsEdDt': '20161231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
디엘종합건설(주)   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)대우건설, 대창건설(주),(주)삼전건설   2011
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)대우건설, 대창건설(주),(주)삼전건설', 'searchRpcdIsBgDt': '20110101', 'searchRpcdIsEdDt': '20111231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
이화공영(주).우람종합건설(주)

In [16]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

품질검사성적서 등록 목록 save compled
>totalCount 3
0 no pageNo
x :  5  g_totalCount :  -997  numOfRows :  1000
명화건설(주)   2019
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '명화건설(주)', 'searchRpcdIsBgDt': '20190101', 'searchRpcdIsEdDt': '20191231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
디엘종합건설(주)   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)경화엔지니어링,(주)장산엔지니어링   2021
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)경화엔지니어링,(주)장산엔지니어링', 'searchRpcdIsBgDt': '20210101', 'searchRpcdIsEdDt': '20211231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Da

In [17]:
resultDfLastMerged.shape

>totalCount 6
0 no pageNo
x :  5  g_totalCount :  -994  numOfRows :  1000
명화건설(주)   2020
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '명화건설(주)', 'searchRpcdIsBgDt': '20200101', 'searchRpcdIsEdDt': '20201231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start


(0, 0)

>totalCount 1
0 no pageNo
x :  7  g_totalCount :  -999  numOfRows :  1000
대림종합건설(주), 대휘건설(주)   2017
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대림종합건설(주), 대휘건설(주)', 'searchRpcdIsBgDt': '20170101', 'searchRpcdIsEdDt': '20171231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  4  g_totalCount :  -999  numOfRows :  1000
(주)대우건설, 대창건설(주),(주)삼전건설   2014
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)대우건설, 대창건설(주),(주)삼전건설', 'searchRpcdIsBgDt': '20140101', 'searchRpcdIsEdDt': '20141231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4
0 no pageNo
x :  1  g_totalCount :  -996  numOfRows :  1000
디엘종합건설(주)   2013
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20130101', 'searchRpcdIsEdDt': '20131231', 'sortField': '

1 page is empty
No Data
흥창건설   2002
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '흥창건설', 'searchRpcdIsBgDt': '20020101', 'searchRpcdIsEdDt': '20021231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
이화공영(주).우람종합건설(주)   2018
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '이화공영(주).우람종합건설(주)', 'searchRpcdIsBgDt': '20180101', 'searchRpcdIsEdDt': '20181231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6
0 no pageNo
x :  1  g_totalCount :  -994  numOfRows :  1000
디엘종합건설(주)   2017
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20170101', 'searchRpcdIsEdDt': '20171231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
대림종합건설(주), 대휘건설(주)   2021
BASEPARAM  {

1 page is empty
No Data
(주효창   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주효창', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  1  g_totalCount :  -998  numOfRows :  1000
디엘종합건설(주)   2021
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '디엘종합건설(주)', 'searchRpcdIsBgDt': '20210101', 'searchRpcdIsEdDt': '20211231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page is empty1 page scraping start

No Data
이화공영(주).우람종합건설(주)   2022
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '이화공영(주).우람종합건설(주)', 'searchRpcdIsBgDt': '20220101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
흥창건설   2007
BASEPARAM  {'serviceKey': 

1 page is empty
No Data
직접기초   2003
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '직접기초', 'searchRpcdIsBgDt': '20030101', 'searchRpcdIsEdDt': '20031231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  5  g_totalCount :  -998  numOfRows :  1000
흥창건설   2011
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '흥창건설', 'searchRpcdIsBgDt': '20110101', 'searchRpcdIsEdDt': '20111231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
대한기업(주)   2004
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대한기업(주)', 'searchRpcdIsBgDt': '20040101', 'searchRpcdIsEdDt': '20041231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
영원건설   2007
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB